In [0]:
import requests
from pyspark.sql.functions import col, asc,desc
import time

url_base = 'https://brasil.io/api/v1/dataset/genero-nomes/grupos/data/?format=json'
TOKEN = "xxxxxxxxx"
headers = {
    "Authorization": f"Token {TOKEN}"
}

resposta = requests.get(url_base, headers=headers).json()
resultados = resposta['results']
quantidade = resposta['count']
qtd_linhas = len(resultados)
total_paginas = int((quantidade + qtd_linhas - 1) / qtd_linhas)


In [0]:
for x in range(2, total_paginas + 1):
    url_paginada = f'{url_base}&page={x}'
    try:
        resposta = requests.get(url_paginada, headers=headers).json()
        novos_resultados = resposta['results']
        resultados.extend(novos_resultados)
        time.sleep(2)
        print(f"Dados da página {x} capturados") 
    except Exception as e:
        print(f'Erro, pagina -> {x}: {e}')
        break




Dados da página 20 capturados
Dados da página 21 capturados
Dados da página 22 capturados
Dados da página 23 capturados
Dados da página 24 capturados
Dados da página 25 capturados
Dados da página 26 capturados
Dados da página 27 capturados
Dados da página 28 capturados
Dados da página 29 capturados
Dados da página 30 capturados
Dados da página 31 capturados
Dados da página 32 capturados
Dados da página 33 capturados
Dados da página 34 capturados
Dados da página 35 capturados
Dados da página 36 capturados
Dados da página 37 capturados
Dados da página 38 capturados
Dados da página 39 capturados
Dados da página 40 capturados
Dados da página 41 capturados
Dados da página 42 capturados
Dados da página 43 capturados
Dados da página 44 capturados
Dados da página 45 capturados
Dados da página 46 capturados
Dados da página 47 capturados
Dados da página 48 capturados
Dados da página 49 capturados
Dados da página 50 capturados


In [0]:
df_nomes = spark.createDataFrame(resultados)



In [0]:
df_nomes_tratados = df_nomes.withColumnRenamed("classification","genero")\
                   .withColumnRenamed("frequency_female","feminino")\
                   .withColumnRenamed("frequency_male","masculino")\
                   .withColumnRenamed("frequency_total","quantidade")\
                   .withColumnRenamed("name","nome")\
                   .withColumnRenamed("ratio","percentual_indice")
df_nomes_tratados.drop('names')


Out[11]: DataFrame[genero: string, feminino: bigint, masculino: bigint, quantidade: bigint, nome: string, percentual_indice: string]

In [0]:



df_nomes_ordenados = df_nomes_tratados.orderBy(col("quantidade").desc(),col("nome").desc())
df_nomes_ordenados = df_nomes_ordenados.drop('names')



In [0]:

df_nomes_ordenados.write.format("delta").mode("overwrite").save("/dbfs/tmp/classificacao_nomes_delta")